# note
- 这些函数不一定对大家都适用，因为html结构不同。所以如果用不了的话，只能自己inspect然后一个一个节点找了。gmail在download前先在gmail的页面gear-setting-conversation view off，目的是一个row只对应一封邮件。为了数据统一好处理，建议大家最后download的结果都是做成[(email,name,title,content,date)]这样的格式。

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"       
import time 
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup     
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [2]:
#这个函数用于直接从gmail登录学校邮箱,如果绑定的是Outlook可能不适用
def login(driver,user,pwd): 
    driver.get('https://gmail.com/')
    driver.find_element_by_id('identifierId').send_keys(user)
    driver.find_element_by_xpath("//div[@id='identifierNext']/content/span").click()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    driver.find_element_by_id("username").send_keys('zwang173')
    driver.find_element_by_id("password").send_keys(pwd)
    driver.find_element_by_name("_eventId_proceed").click()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    driver.find_element_by_xpath("//div[@id='view_container']/descendant::content/span").click()
    driver.switch_to.window(driver.window_handles[0])
    return driver

In [3]:
#这个函数直接用于登录gmail,其他邮箱可能不适用
def login_gmail(driver,user,pwd):    
    driver.get('https://gmail.com/')
    driver.find_element_by_id('identifierId').send_keys(user)
    driver.find_element_by_xpath("//div[@id='identifierNext']/content/span").click()
    time.sleep(2)
    driver.find_element_by_name('password').send_keys(pwd)
    driver.find_element_by_xpath("//div[@id='passwordNext']/content/span").click()
    return driver

In [ ]:
driver=login_gmail(webdriver.Chrome(),'https://gmail.com/','your username','your pwd')

In [14]:
#这个函数用于提取邮件的邮件域，标题，内容及时间，spam若为true则提取垃圾邮件反之提取正常邮件
#函数有可能出现卡顿或报错，可以试着用time.sleep()解决，或者多运行几遍(测试了很多遍，报错多运行几遍一定可以解决)
#我不确定每个人的gmail的html是否都相同，若有不同只需要改其中的id或者class，其他邮箱可能要改结构
#该结构的话尽量不要改iframe，邮箱通常会拒绝访问
#提取的结果为列表，其中元素为元组
def get_data(driver,spam=False):
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lst = []
    n = 0
    
    if spam == True:
        time.sleep(3)
        #这个地方的span节点不太稳定，id有时会变，所以有时会报错，能改进希望改进，目前多跑几次就可以顺利运行
        driver.find_element_by_xpath("//span[@id=':hs']/span").click()
        driver.find_element_by_xpath("//div[@id=':i2']/descendant::span").click()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tbody = soup.select('div#:1 table#:k5 tbody tr')
        page_number = int(int([i.get_text() for i in soup.select('div#:zm span.ts')][-1].replace(',','')) / 50)
        time.sleep(2)
    else:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tbody = soup.select('div#:1 table#:20 tbody tr')
        page_number = int(int([i.get_text() for i in soup.select('div#:hh span.ts')][-1].replace(',','')) / 50)
        time.sleep(2)

    while n <= page_number: 
        
        for idx, tr in enumerate(tbody):
            name = None
            email= None
            title = None
            date = None
            
            td_sender = tr.select('td.yX.xY span.bA4')[0].select_one('span')
            if td_sender != []:
                email = td_sender.attrs['email']
                name = td_sender.attrs['name']           

            td_title = tr.select('td.xY.a4W span.bog')
            if td_title != []:
                title = td_title[0].get_text()

            td_content = tr.select('td.xY.a4W span.y2')
            if td_content != []:
                content = td_content[0].get_text()
            
            td_date = tr.select('td.xW.xY span')
            if td_date != []:
                date = td_date[0].attrs['title']    
                
            lst.append((email,name,title,content,date))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        id_click = soup.select('div.ar5.J-J5-Ji span.Di div')[-1].attrs['id']
        time.sleep(1)
        if n < page_number:
            driver.find_element_by_id(id_click).click()
            driver.switch_to.window(driver.window_handles[0])
        n += 1
        time.sleep(1)
    driver.quit()
    return lst

In [7]:
get_data(login(webdriver.Chrome(),user,pwd),spam=True)

[('workinentertainment@workinentertainment.com',
  'WorkInEntertainment.',
  'Zixuan: Your Job Alert',
  '\xa0-\xa0The latest and greatest entertainment jobs based on your settings: Update Your Job Alert Settings Hello Zixuan! Below are the results of your email job alert: Job Title Location Audio Engineer -',
  'Mon, Mar 4, 2019, 10:35 AM'),
 ('invitations@linkedin.com',
  'ziyao wang',
  'Zixuan(William), please add me to your LinkedIn network',
  "\xa0-\xa0Hi Zixuan(William), I'd like to join your LinkedIn network. LinkedIn Zixuan(William) Wang Hi Zixuan(William), I'd like to join your LinkedIn network. ziyao wang Assistant project manager at",
  'Sun, Mar 3, 2019, 11:31 PM'),
 ('jobalerts-noreply@linkedin.com',
  'LinkedIn Job Alerts',
  "Zixuan(William): 58 new jobs for 'data intern'",
  "\xa0-\xa0See results for your search query 'data intern' LinkedIn Zixuan(William) Wang Your job search alert “data intern” 58 jobs matching “Intern” SSI Schäfer Dunnage Intern SSI Schäfer · Charl

In [15]:
data_s=get_data(login(webdriver.Chrome(),user,pwd),spam=True)

In [18]:
len(data_s)

250

In [17]:
data_h=get_data(login(webdriver.Chrome(),user,pwd))

In [19]:
len(data_h)

1300